# breadboard for census community resilience estimates
# https://www.census.gov/programs-surveys/community-resilience-estimates/data/datasets.html
"C:\code\capstone\data\census_resiliance.csv"

FIPS format (11 digits): SSCCCTTTTTT

SS = 2-digit state code
CCC = 3-digit county code
TTTTTT = 6-digit (census) tract code

In [ ]:
# import libraries
# NOTE: global_vars should be edited to include local paths and credentials before use.
# If global_vars.py is created in the root dir remove the ignore/ prefix in the import statement below.
import ignore.global_vars as gv
import db_tools as dbt
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import urllib.request
import re


In [ ]:
df = pd.read_csv(gv.DATA_PATHS["census_resilience"], encoding='latin-1')
df.head()

,GEO_ID,STATE,COUNTY,TRACT,NAME,GEO_LEVEL,WATER_TRACT,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM
0,1400000US01001020100,1,1,20100,"Census Tract 201, Autauga County, Alabama",Tract,NaN,1815,560,228,30.85,12.56,871,239,47.99,13.17,384,205,21.16,11.29
1,1400000US01001020200,1,1,20200,"Census Tract 202, Autauga County, Alabama",Tract,NaN,2004,720,262,35.93,13.07,896,269,44.71,13.42,388,222,19.36,11.08
2,1400000US01001020300,1,1,20300,"Census Tract 203, Autauga County, Alabama",Tract,NaN,3313,817,404,24.66,12.19,1831,405,55.27,12.22,665,317,20.07,9.57
3,1400000US01001020400,1,1,20400,"Census Tract 204, Autauga County, Alabama",Tract,NaN,4343,1301,522,29.96,12.02,2274,523,52.36,12.04,768,407,17.68,9.37
4,1400000US01001020501,1,1,20501,"Census Tract 205.01, Autauga County, Alabama",Tract,NaN,4442,1979,515,44.55,11.59,1670,515,37.60,11.59,793,392,17.85,8.82


In [40]:
print(df.shape)
print(df.info())

(87611, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87611 entries, 0 to 87610
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GEO_ID       87611 non-null  object 
 1   STATE        87611 non-null  int64  
 2   COUNTY       87611 non-null  int64  
 3   TRACT        87611 non-null  int64  
 4   NAME         87611 non-null  object 
 5   GEO_LEVEL    87611 non-null  object 
 6   WATER_TRACT  320 non-null    float64
 7   POPUNI       87611 non-null  int64  
 8   PRED0_E      87611 non-null  int64  
 9   PRED0_M      87611 non-null  int64  
 10  PRED0_PE     87611 non-null  float64
 11  PRED0_PM     87611 non-null  float64
 12  PRED12_E     87611 non-null  int64  
 13  PRED12_M     87611 non-null  int64  
 14  PRED12_PE    87611 non-null  float64
 15  PRED12_PM    87611 non-null  float64
 16  PRED3_E      87611 non-null  int64  
 17  PRED3_M      87611 non-null  int64  
 18  PRED3_PE     87611 non-null  float

In [8]:
df_tract = df.copy()
df_tract['TRACT'] = df_tract['TRACT'].astype(str)
df_tract['TRACT'].str.len().describe()

count    87611.000000
mean         4.972298
std          1.172546
min          1.000000
25%          4.000000
50%          5.000000
75%          6.000000
max          6.000000
Name: TRACT, dtype: float64

In [9]:
df['GEO_LEVEL'].unique()

array(['Tract', 'County', 'State', 'US'], dtype=object)

In [10]:
df.groupby('GEO_LEVEL').head(5)

,GEO_ID,STATE,COUNTY,TRACT,NAME,GEO_LEVEL,WATER_TRACT,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM
0,1400000US01001020100,1,1,20100,"Census Tract 201, Autauga County, Alabama",Tract,NaN,1815,560,228,30.85,12.56,871,239,47.99,13.17,384,205,21.16,11.29
1,1400000US01001020200,1,1,20200,"Census Tract 202, Autauga County, Alabama",Tract,NaN,2004,720,262,35.93,13.07,896,269,44.71,13.42,388,222,19.36,11.08
2,1400000US01001020300,1,1,20300,"Census Tract 203, Autauga County, Alabama",Tract,NaN,3313,817,404,24.66,12.19,1831,405,55.27,12.22,665,317,20.07,9.57
3,1400000US01001020400,1,1,20400,"Census Tract 204, Autauga County, Alabama",Tract,NaN,4343,1301,522,29.96,12.02,2274,523,52.36,12.04,768,407,17.68,9.37
4,1400000US01001020501,1,1,20501,"Census Tract 205.01, Autauga County, Alabama",Tract,NaN,4442,1979,515,44.55,11.59,1670,515,37.60,11.59,793,392,17.85,8.82
84415,0500000US01001,1,1,0,"Autauga County, Alabama",County,NaN,60183,23534,2790,39.10,4.64,24255,2851,40.30,4.74,12394,2306,20.59,3.83
84416,0500000US01003,1,3,0,"Baldwin County, Alabama",County,NaN,251949,91386,7887,36.27,3.13,107840,8199,42.80,3.25,52723,6888,20.93,2.73
84417,0500000US01005,1,5,0,"Barbour County, Alabama",County,NaN,22085,4988,1080,22.59,4.89,9758,1204,44.18,5.45,7339,1123,33.23,5.08
84418,0500000US01007,1,7,0,"Bibb County, Alabama",County,NaN,20223,6880,1051,34.02,5.20,8050,1109,39.81,5.48,5293,967,26.17,4.78
84419,0500000US01009,1,9,0,"Blount County, Alabama",County,NaN,59729,18973,2245,31.77,3.76,26090,2367,43.68,3.96,14666,2036,24.55,3.41


In [ ]:
# verify no useful FIPS codes in non-tract rows
# describe statement should show no >= 2 digit values
df_not_tract = df[df['GEO_LEVEL'] != 'Tract'].copy()
df_not_tract['TRACT'] = df_not_tract['TRACT'].astype(str)
df_not_tract = df_not_tract.reset_index(drop=True)
df_not_tract['TRACT'].str.len().describe()

count    3196.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: TRACT, dtype: float64

In [ ]:
# Filter for only 6-digit TRACT values
df_tract_only = df[df['GEO_LEVEL'] == 'Tract'].copy()
print(df_tract_only.shape)
df_tract_only.head()

(84415, 20)


,GEO_ID,STATE,COUNTY,TRACT,NAME,GEO_LEVEL,WATER_TRACT,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM
0,1400000US01001020100,1,1,20100,"Census Tract 201, Autauga County, Alabama",Tract,NaN,1815,560,228,30.85,12.56,871,239,47.99,13.17,384,205,21.16,11.29
1,1400000US01001020200,1,1,20200,"Census Tract 202, Autauga County, Alabama",Tract,NaN,2004,720,262,35.93,13.07,896,269,44.71,13.42,388,222,19.36,11.08
2,1400000US01001020300,1,1,20300,"Census Tract 203, Autauga County, Alabama",Tract,NaN,3313,817,404,24.66,12.19,1831,405,55.27,12.22,665,317,20.07,9.57
3,1400000US01001020400,1,1,20400,"Census Tract 204, Autauga County, Alabama",Tract,NaN,4343,1301,522,29.96,12.02,2274,523,52.36,12.04,768,407,17.68,9.37
4,1400000US01001020501,1,1,20501,"Census Tract 205.01, Autauga County, Alabama",Tract,NaN,4442,1979,515,44.55,11.59,1670,515,37.60,11.59,793,392,17.85,8.82


In [35]:
# Drop unused columns
# STATE, COUNTY, NAME, GEO_LEVEL, TRACT, WATER_TRACT
df_tract_only = df_tract_only.drop(columns=['STATE', 'COUNTY', 'NAME', 'GEO_LEVEL', 'TRACT', 'WATER_TRACT'], errors='ignore')
df_tract_only.head()

,GEO_ID,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM,FIPS
0,1400000US01001020100,1815,560,228,30.85,12.56,871,239,47.99,13.17,384,205,21.16,11.29,01001020100
1,1400000US01001020200,2004,720,262,35.93,13.07,896,269,44.71,13.42,388,222,19.36,11.08,01001020200
2,1400000US01001020300,3313,817,404,24.66,12.19,1831,405,55.27,12.22,665,317,20.07,9.57,01001020300
3,1400000US01001020400,4343,1301,522,29.96,12.02,2274,523,52.36,12.04,768,407,17.68,9.37,01001020400
4,1400000US01001020501,4442,1979,515,44.55,11.59,1670,515,37.60,11.59,793,392,17.85,8.82,01001020501


In [36]:
df_tract_only['FIPS'] = df_tract_only['GEO_ID'].str.split('US').str[1]
df_tract_only.head()

,GEO_ID,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED12_E,PRED12_M,PRED12_PE,PRED12_PM,PRED3_E,PRED3_M,PRED3_PE,PRED3_PM,FIPS
0,1400000US01001020100,1815,560,228,30.85,12.56,871,239,47.99,13.17,384,205,21.16,11.29,01001020100
1,1400000US01001020200,2004,720,262,35.93,13.07,896,269,44.71,13.42,388,222,19.36,11.08,01001020200
2,1400000US01001020300,3313,817,404,24.66,12.19,1831,405,55.27,12.22,665,317,20.07,9.57,01001020300
3,1400000US01001020400,4343,1301,522,29.96,12.02,2274,523,52.36,12.04,768,407,17.68,9.37,01001020400
4,1400000US01001020501,4442,1979,515,44.55,11.59,1670,515,37.60,11.59,793,392,17.85,8.82,01001020501


In [ ]:
# Drop GEO_ID column now that FIPS has been extracted, check memory usage
df_clean = df_tract_only.drop(columns=['GEO_ID'], errors='ignore')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84415 entries, 0 to 84414
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POPUNI     84415 non-null  int64  
 1   PRED0_E    84415 non-null  int64  
 2   PRED0_M    84415 non-null  int64  
 3   PRED0_PE   84415 non-null  float64
 4   PRED0_PM   84415 non-null  float64
 5   PRED12_E   84415 non-null  int64  
 6   PRED12_M   84415 non-null  int64  
 7   PRED12_PE  84415 non-null  float64
 8   PRED12_PM  84415 non-null  float64
 9   PRED3_E    84415 non-null  int64  
 10  PRED3_M    84415 non-null  int64  
 11  PRED3_PE   84415 non-null  float64
 12  PRED3_PM   84415 non-null  float64
 13  FIPS       84415 non-null  object 
dtypes: float64(6), int64(7), object(1)
memory usage: 11.7+ MB


In [38]:
df_tract_only.memory_usage(deep=True)


Index        2788896
GEO_ID       5824635
POPUNI        675320
PRED0_E       675320
PRED0_M       675320
PRED0_PE      675320
PRED0_PM      675320
PRED12_E      675320
PRED12_M      675320
PRED12_PE     675320
PRED12_PM     675320
PRED3_E       675320
PRED3_M       675320
PRED3_PE      675320
PRED3_PM      675320
FIPS         5064900
dtype: int64

In [42]:
# load to db when ready
dbt.load_data(df_clean, 'census_resilience', if_exists='replace')

Created SQLAlchemy engine for disaster_db
Data loaded successfully into census_resilience
